# init

In [1]:
from pymongo import MongoClient

client = MongoClient()
db = client.izidb

In [27]:
from scripts import izi
reload(izi)

n_topics = 100

--------------------> english word frequencies loaded


# get files list

In [3]:
root = u"../izi_data/"
filelist = izi.getFileList(root)
print len(filelist)
example =  filelist[0]
print example[len(root):]

195
1412 Standard English Review-en-ru-T.mxliff


# define functions

In [37]:
def insertDoc(path):
    document = dict()
    document['title'] = path[len(root):]
    full_text = izi.loadText(path)
    tokens = izi.tokenize( full_text)
    document['full_text'] = full_text
    document['tokens'] = tokens
    topics =  izi.topicsFromTokens(izi.tokenize(full_text))
    semantic_vec = [0.] * n_topics
    for i in topics:
        semantic_vec[i[0]] = i[1]
    document['semantic_vec'] = semantic_vec
    current_id = db.documents.save(document)
    # create links
    cursor = db.documents.find()
    for doc in cursor:
    	y = doc['semantic_vec']
    	y_id = doc["_id"]
    	if y_id != current_id:
	    	s = izi.getSimilarity( semantic_vec, y)
	    	db.similarities.insert({'source': current_id , 'target': y_id, 'value': s})

In [38]:
(insertDoc(example))

In [39]:
# cursor = db.documents.find_one({ "_id": id})
# for document in cursor:
#     print document

# add all documents

In [48]:
fails = []
i = 0
maxx = 4
for f in filelist:
    if i < maxx:
        try:
            insertDoc(f)
            print i,
        except:
            fails.append(f)
            print "#",

        i += 1
    
print
print
print "%s fails" %len(fails)

0 1 2 3

0 fails


In [41]:
for f in fails:
    print f

# tests

In [32]:
cursor = db.documents.find_one()
for document in cursor:
    print document
    
    
x = cursor["_id"]
print str(x)

db.documents.find_one({ "_id": x})["_id"]

TypeError: 'NoneType' object is not iterable

In [31]:
db.similarities.find_one()

# reset collections

In [47]:
db.restaurants.drop()
db.documents.drop()
db.similarities.drop()